<img src="../../img/logo-bdc.png" align="right" width="64" />

# <span style="color:#336699">SpatioTemporal Asset Catalog (STAC)</span>
<hr style="border:2px solid #0077b9;">

The [**S**patio**T**emporal **A**sset **C**atalog (STAC)](https://stacspec.org/) is a specification created through the colaboration of several organizations intended to increase satellite image search interoperability.

The diagram depicted in the picture contains the most important concepts behind the STAC data model:

<img src="../../img/stac/stac-model.png" width="480" />
<br/>
<b>Figura 1</b> - STAC model.

# 1. STAC API
<hr style="border:1px solid #0077b9;">

The STAC API is the dynamic version of a STAC. It returns `STAC Catalog`, `Collection`, `Item`, `ItemCollection` objects through several *endpoints*.

The description of the concepts below are adapted from the [STAC Specification](https://github.com/radiantearth/stac-spec):

- **Item**: a `STAC Item` is the atomic unit of metadata in STAC, providing links to the actual `assets` (including thumbnails) that they represent. It is a `GeoJSON Feature` with additional fields for things like time, links to related entities and mainly to the assets. According to the specification, this is the atomic unit that describes the data to be discovered in a `STAC Catalog` or `Collection`.

- **Asset**: a `spatiotemporal asset` is any file that represents information about the earth captured in a certain space and time.


- **Catalog**: provides a structure to link various `STAC Items` together or even to other `STAC Catalogs` or `Collections`.


- **Collection:** is a specialization of the `Catalog` that allows additional information about a spatio-temporal collection of data.

For running the examples in this Jupyter Notebook you will need to install the [STAC client for Python](https://github.com/brazil-data-cube/stac.py). To install it from the Brazil Data Cube's GitHub repository, you can use `pip` with the following command:

In [ ]:
!pip install stac.py

Let's import the `stac` package, as follows:

In [ ]:
import stac

After that, you can check the installed version of stac package:

In [ ]:
stac.__version__

Then, create a `STAC` object called `service` attached to the service address, which will allow us to comunicate to the given `STAC` service.

In [ ]:
service = stac.STAC("http://brazildatacube.dpi.inpe.br/stac/")

# 2. Listing the Available Data Products
<hr style="border:1px solid #0077b9;">

calling the `STAC` object retrieves the image collections and data cube collections available in the server.

In [ ]:
service

# 3. Retrieving the Metadata of a Data Product
<hr style="border:1px solid #0077b9;">

The `collection` method returns information about a given image or data cube collection identified by its name. In this example we are retrieving inormation about the datacube collection `CB4_64_16D_STK-1`:

In [ ]:
collection = service.collection('CB4_64_16D_STK-1')
collection

# 4. Retrieving data
<hr style="border:1px solid #0077b9;">

The `get_items` method returns a consult given a `bbox` and `datetime`:

In [ ]:
items = collection.get_items(filter={'bbox':'-46.62597656250001,-13.19716452328198,-45.03570556640626,-12.297068292853805',
                       'datetime':'2018-08-01/2019-07-31', 
                       'limit':10})
items

In [ ]:
first_date_blue_url = items['features'][0]['assets']['BAND13']['href']
first_date_green_url = items['features'][0]['assets']['BAND14']['href']
first_date_red_url = items['features'][0]['assets']['BAND15']['href']
first_date_blue_url

In [ ]:
import numpy
import rasterio

from matplotlib import pyplot as plt
from rasterio.windows import Window
%matplotlib inline

Let's use `Window` from the `rasterio` package to read a subset of the original image, in this case from row 0 to row 3000 and column 0 to column 3000

In [ ]:
with rasterio.open(first_date_red_url) as dataset:
    red = dataset.read(1, window=Window(0, 0, 3000, 3000)) # Window(col_off, row_off, width, height)

with rasterio.open(first_date_green_url) as dataset:
    green = dataset.read(1, window=Window(0, 0, 3000, 3000)) # Window(col_off, row_off, width, height)
    
with rasterio.open(first_date_blue_url) as dataset:
    blue = dataset.read(1, window=Window(0, 0, 3000, 3000)) # Window(col_off, row_off, width, height)

# 5. Visualizing the Images
<hr style="border:1px solid #0077b9;">

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(12, 4))
ax1.imshow(red, cmap='gray')
ax2.imshow(green, cmap='gray')
ax3.imshow(blue, cmap='gray')
fig.show()

In [ ]:
def normalize(array):
    """Normalizes numpy arrays into scale 0.0 - 1.0"""
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

In [ ]:
rgb = numpy.dstack((normalize(red), normalize(green), normalize(blue)))
plt.imshow(rgb)

# 6. Retrieving data using latlong
<hr style="border:1px solid #0077b9;">

A function to use latlong instead of the image row and column is defined next:

In [ ]:
!pip install pyproj

In [ ]:
from math import floor, ceil
from pyproj import Proj

def longlat2window(lon, lat, dataset):
    """
    Args:
        lon (tuple): Tuple of min and max lon
        lat (tuple): Tuple of min and max lat
        dataset: Rasterio dataset

    Returns:
        rasterio.windows.Window
    """
    p = Proj(dataset.crs)
    t = dataset.transform
    xmin, ymin = p(lon[0], lat[0])
    xmax, ymax = p(lon[1], lat[1])
    col_min, row_min = ~t * (xmin, ymin)
    col_max, row_max = ~t * (xmax, ymax)
    return Window.from_slices(rows=(floor(row_max), ceil(row_min)),
                              cols=(floor(col_min), ceil(col_max)))

In [ ]:
w = -45.90
n = -12.6
e = -45.40
s = -12.90

with rasterio.open(first_date_red_url) as dataset_red:
    red = dataset_red.read(1, window = longlat2window((w,e), (s,n), dataset_red))
    plt.imshow(red, cmap='gray')
    plt.show()

# 7. Visualizing Composed Images
<hr style="border:1px solid #0077b9;">

A color composition can also be made:

In [ ]:
with rasterio.open(first_date_blue_url) as dataset_blue:
    blue = dataset_blue.read(1, window = longlat2window((w,e), (s,n), dataset_blue))
with rasterio.open(first_date_green_url) as dataset_green:
    green = dataset_green.read(1, window = longlat2window((w,e), (s,n), dataset_green))

In [ ]:
rgb = numpy.dstack((normalize(red), normalize(green), normalize(blue)))
plt.imshow(rgb)

# 8. References
<hr style="border:1px solid #0077b9;">

- [Spatio Temporal Asset Catalog Specification](https://stacspec.org/)


- [Brazil Data Cube Python Client Library for STAC Service - GitHub Repository](https://github.com/brazil-data-cube/stac.py)